In [1]:
cd ..

/people/gerald/Documents/repositories/continual_learning_of_long_topic


In [2]:
from lire.dataset import msmarco
import os
os.environ['DATA_FOLDER'] = './storage'

## Loading the dataset
To load the dataset call the MSMarcoRankingDataset, notice it will need pyserini correctly installed. If an error occurs in this example you probably should update your java version.

In [5]:
# it can be costly in term of memory ~ 20 Go if load_triplets is set to true, first time dataset is created can be quiet long
dataset = msmarco.MSMarcoRankingDataset(
    'ressources/V1.0/MsMarco-continual-medium/', 
    load_triplets=True, 
    bm25_compute=True
)

Loading qrels
Loading documents
Loading queries
loading triplet
loading scoreddocs
loading scoreddocs


In [9]:
dataset.set_split('val')
dataset.triplets_collection = None
#dataset.set_n_sample_negative(2) # if using negative does not work for the dev (test set)

In [10]:
dataset[0]

IndexError: list index out of range

In [5]:
dataset[156]

{'qid': '1141090',
 'qtxt': 'where is hogwarts a real place',
 'pdid': ['2434703'],
 'pdtxt': ['In the novels, the Hogwarts School of Witchcraft and Wizardry is located in the Scottish Highlands. Few places in the world have the mystique that the Highlands possess, and while it naturally seems like the perfect place for the real Hogwarts, sadly you will not be able to find such an edifice in this region. Real Hogwarts'],
 'ndid': [8485702, 1569152],
 'ndtxt': ['Learn what makes Donalds , South Carolina a best place to live, including information about real estate, schools, employers, things to do and more. Learn what makes Donalds , South Carolina a best place to live, including information about real estate, schools, employers, things to do and more.',
  'The best place to start your real estate search. Trulia is an all-in-one real estate site that gives you the local scoop about homes for sale, apartments for rent, neighborhood insights, and real estate markets and trends to help you

In [6]:
dataset.set_task(1)
print("Example of queries for the task 0 :\n", [dataset[i]['qtxt'] for i in range(10)])

Example of queries for the task 0 :
 ['good temperature to keep food warm in oven', 'how long does it take a chicken to hatch after pipping', 'home fries calories per ounce', 'what smoker temperature for barbecue chicken', 'how long does bbq chicken take to cook', 'sonic boneless chicken wings how many carbs', 'how long does it take to bake chicken legs', 'how long to bake chicken cubes in oven', 'how long to bake chicken thighs in the oven', 'at what temperature do i roast a chicken in the oven']


In [7]:
dataset.set_task(12)
print("Example of queries for the task 10 :\n", [dataset[i]['qtxt'] for i in range(10)])

Example of queries for the task 10 :
 ['subscription to pubmed typically cost', 'salary level of an acute care nurse practitioner', 'how many payment years in medicaid', 'how much do doctors make giving botox', 'can i make money as an independent respiratory therapist', 'what c-peptide medicare requirement', 'what organizations support professional medical assistants', 'what is a medical records technician salary', 'medical technologist average salary', 'vesicare medication cost']


## Transform the input (exemple for MonoT5)
 Here an example for transforming the input for monoT5 `Query:%s Document:%s Relevant:`

In [8]:
import os 

In [53]:
from lire.trainer import monoT5
from lire.trainer.lire_experiment import dict_collate_fn

In [37]:
dataset.set_n_sample_negative(1)
dt = monoT5.MonoT5TrainingDataset(dataset)

In [38]:
# positive input
dt[5]['ninput']

'Query:what c-peptide medicare requirement Document:Because the Medicare ambulance benefit is a transport benefit, if no transport of a Medicare beneficiary occurs, then there is no Medicare-covered service. In general, if the beneficiary dies before being transported, then no Medicare payment may be made. Relevant:'

In [39]:
# negative input
dt[5]['pinput']

'Query:what c-peptide medicare requirement Document:C peptide test and Medicare. To qualify for a pump under medicare. The test is designed to disqualify T2. Glucose must be 225 or more, c-peptide 110% or less of the minimum normal at your lab, simultaneous. -Lloyd. Actually, I believe that medicare requires that CMS has determined that fasting C-peptide levels will only be considered valid when a concurrently obtained fasting glucose is < 225 mg/dL. Relevant:'

In [40]:
trainer = monoT5.DDPMonoT5(hyperparameters={'log_folder': '~/test', 'dataset':'topics', 'switch':'' })

DDPLIReExperiment using master node  localhost on 8879
checking data_folder
Logs will be find at  /people/gerald/test/DDPMonoT5-topics/run_3


In [41]:
from torch.utils.data import DataLoader

In [51]:
dt.dataset.queries_collection.loc['504888']

text    subscription to pubmed typically cost
Name: 504888, dtype: object

In [57]:
dl = DataLoader(dt , batch_size=2, collate_fn=dict_collate_fn)

In [58]:
next(iter(dl))

{'qid': ['504888', '491785'],
 'qtxt': ['subscription to pubmed typically cost',
  'salary level of an acute care nurse practitioner'],
 'pdid': [['7006123', '7006129'], ['6284118']],
 'pdtxt': [["Subscription-based pricing is increasingly being used for cloud computing. In a subscription-based model, cloud customers typically pay upfront, prior to receiving access to cloud services. Prices are often based on the subscription's length and a longer subscription often translates to a lower cost.",
   "subscription-based pricing model. A subscription-based pricing model is a payment structure that allows a customer or organization to purchase or subscribe to a vendor's IT services for a specific period of time for a set price. Subscribers typically commit to the services on a monthly or annual basis."],
  ['Acute care nurse practitioner with less than 1 year of experience collects an average yearly pay from $50,048 to $81,094 while an acute care nurse practitioner practicing for 1 to 5 ye

In [62]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
trainer.tokenizer = T5Tokenizer.from_pretrained('t5-small')
trainer.device = 'cpu'
trainer.batch_transform(next(iter(dl)))

{'input_ids': tensor([[    3, 27569,    10,  7304, 11830,    12, 11943,  2726,  3115,   583,
          11167,    10, 25252, 11830,    18,   390,  5769,    19,  5684,   271,
            261,    21,  3126, 10937,     5,    86,     3,     9,  7644,    18,
            390,   825,     6,  3126,   722,  3115,   726, 25440,     6,  1884,
             12,  4281,   592,    12,  3126,   364,     5, 16511,    33,   557,
              3,   390,    30,     8,  7644,    31,     7,  2475,    11,     3,
              9,  1200,  7644,   557,     3, 29213,    12,     3,     9,  1364,
            583,     5, 31484,    17,    10,     1,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    